# Operators

In [1]:
from fn import op

## curry

In [2]:
def myadd(a):
    def myadd(b):
        return a+b
    return myadd

op.curry(myadd, 1, 2)

3

## apply

In [3]:
import operator
op.apply(operator.add, [2,8])

10

## flip

In [4]:
op.flip(operator.sub)(5,3)

-2

In [5]:
from fn import _ as X
op.flip(X - X)(5,3)

-2

## zipwith

In [6]:
import itertools

zipper = op.zipwith(operator.sub)
list(zipper([1,2,3], itertools.repeat(5)))

[-4, -3, -2]

## foldl/foldr

In [7]:
op.foldl(operator.sub)([1,2,5])
# foldl是按next()顺序从参数中逐个取出来执行
# 首先取出 (1,2), 算 1-2 = -1
# 然后就把前一步结果(-1)当作参数, 发现还缺一个参数，就把5取出来；算 -1 - 5 = -6

-6

In [8]:
op.foldr(operator.sub)([1,2,5])
# foldr是按reversed(next())顺序从参数中逐个取出来执行
# 首先取出 (2,5), 算 2-5 = -3
# 然后就把前一步结果(-3)当作参数, 发现还缺一个参数，就把1取出来（注意旧参数位置仍在右边）；算 1 - (-3) = 4

4

In [9]:
op.foldl(operator.sub, init=9)([1,2,5])
# 已经有了init参数9，还缺一个,取出 1；算 9-1=8，注意9的位置，对于 foldl来说，相当于在序列最左边插一个9
# 然后依次是：8-2=6
# 6-5=1

1

In [10]:
op.foldr(operator.sub, init=9)([1,2,5])
# foldr是按reversed(next())顺序从参数中逐个取出来执行
# 已经有了init参数9，还缺一个,取出 5；算 5-9 = -4 注意init参数的位置，对于 foldr来说，就相当于在序列右边多插一个9
# 然后依次是：2 - (-4) = 6
# 1 - 6 = -5

-5

In [11]:
f1 = lambda x: str(x)
f2 = lambda x: x**2
f3 = lambda x: x+10

op.foldr(op.call, init=10)([f1, f2, f3])
# 这三个函数都只需要刚好一个参数；可以看成是从右向左的composite

'400'

## unfold

`op.unfold` 接受一个函数`f`，对其进行装饰，生成一个展开器; 展开器的作用是接受一个scalar，生成一个Stream

`f`必须接受一个参数`arg1`，返回两个参数`res1`, `res2`
* `arg1` 初始参数
* `res1` 用于向Stream中插入的结果
* `res2` 用作下次计算的cursor

另外，`f`也可以返回`None`表示Stream结束

In [12]:
f = lambda x: (x-1, x-2) if x>1 else None
count_down = op.unfold(f)
list(count_down(10))

# 第一次输入10，返回 (9,8)；其中的9被插入的Stream中，而8作为下次的cursor（输入）
# 第二次输入8，返回（7,6）；其中7被插入，6被当作下次的cursor
# ... 6
# ...
# ... 2, (1,0)；其中1被插入，0被当作下次的cursor
# (0 > 1) 不满足，返回的是None，Stream结束

[9, 7, 5, 3, 1]

In [13]:
# 直接写成装饰器语法
@op.unfold
def f2(x):
    if x>1:
        return (x-1, x-2)
    else:
        return None
list(f2(10))

[9, 7, 5, 3, 1]

# Underscore

## Identity 常函数

In [14]:
f = X(10)
f

10

## Arithmetic 基础算术

In [15]:
f = X+2
f(3)

5

## 还支持 乘方/取模/移位/取反

In [16]:
(X ** 2)(3)

9

In [17]:
(X % 2)(3)

1

In [18]:
(X << 2)(8)

32

## multiple X

In [19]:
(X - X)(5,3) # 名字都叫X，但是按顺序逐个代入的

2

## bitwise operation

In [20]:
(1 & X)(0)

0

In [21]:
(1 & X)(1)

1

## property/attribute getter

In [22]:
class Foo(object):
    def __init__(self):
        self.attr1 = 10
        self.attr2 = 'mystr'
    @property
    def prop1(self):
        return self.attr2.upper()
    def method1(self):
        return self.attr2.capitalize()
    
foo = Foo()
(X.attr1*-2)(foo)

-20

In [23]:
(X.attr2)(foo)

'mystr'

In [24]:
(X.prop1)(foo)

'MYSTR'

In [25]:
(X.method1())(foo) # 不支持这样用method

ArityError: getattr(_, 'method1') expected 1 arguments, got 0

## method

In [26]:
f = X.call('method1')
f(foo)

'Mystr'

In [27]:
f = X.call('split','-')
f('abc-def')

['abc', 'def']

In [28]:
d = {'num':32}
f = X.call('update', num=42)
f(d)
d

{'num': 42}

## slice

In [29]:
f = X[:2]
f(range(10))

range(0, 2)

In [30]:
f = X[X * -1] # 注意多参数的slice只支持切出scalar，不支持片段slice
f(range(10), 2)

8

## toString

In [31]:
# underscore 表达式可以直观地to string

f = X**2 + X**2
str(f)

'(x1, x2) => ((x1 ** 2) + (x2 ** 2))'

In [32]:
# 自动处理优先级

f = X + X * X
str(f)

'(x1, x2, x3) => (x1 + (x2 * x3))'

# Composition

## basic composition

In [33]:
f = X*2
g = X+10
h = X*10

(F(f) << g << h)(5) # [(?*10) + 10] * 2
# 注意 << 是向内复合，好记：箭头从右向左指，表示先执行右边的，再代入左边的函数

NameError: name 'F' is not defined

In [ ]:
(F(f) << F(g) << F(h))(5) # 多个F对象也可以复合

## partial

In [34]:
f = F(operator.sub, 10)
f(3)

NameError: name 'F' is not defined

## pipe composition 向外复合

In [35]:
f = F(X**2) >> (X+10)
f(5)

NameError: name 'F' is not defined

## partial pipe

In [36]:
from fn import iters
f = F() >> (iters.filter, X%2==0) >> sum
f(iters.range(5))

NameError: name 'F' is not defined

# Iterators

## take/drop

In [37]:
list(iters.take(3, range(10)))

[0, 1, 2]

In [38]:
list(iters.take(13, range(10))) # Stream的长度也是有限的, take过长无效

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [39]:
list(iters.drop(3, range(10))) # drop(3,) 就是把前三个元素 [0,1,2] 从Stream中去掉

[3, 4, 5, 6, 7, 8, 9]

In [40]:
list(iters.takelast(3, range(10))) # 从Stream中取出倒数前3个元素

[7, 8, 9]

In [41]:
list(iters.droplast(3, range(10))) # 从Stream中去掉倒数前3个元素

[0, 1, 2, 3, 4, 5, 6]

## ~~`iters.first_true`()~~
跟文档不符，实际上并没有这个方法

In [42]:
import fn
fn.__version__

'0.4.3'

## head/tail

In [43]:
iters.head([3,5,6])

3

In [44]:
print(None == iters.head([]))

True


## consume

In [45]:
my_iterator = iter(range(5,10)) # 注意要弄成iterator，直接写range无效
iters.consume(my_iterator, n=2) # consume是安静地向前推动Iterator, n=None时表示向前推到头
list(my_iterator)

[7, 8, 9]

## nth

In [46]:
iters.nth(range(5,10), 2)

7

In [47]:
iters.nth(range(5,10), 99, default='nan')

'nan'

## first==head, tail==rest

In [48]:
print(iters.head == iters.first) # 是同一个函数

True


In [49]:
print(iters.tail == iters.rest) # 是同一个函数

True


## second

In [50]:
iters.second([2,5,9])

5

## ffirst ( flatten-first)

In [51]:
iters.ffirst([[1,2],[3,4]])

1

In [52]:
iters.first([[1,2],[3,4]])

[1, 2]

## compact

In [53]:
res = iters.compact([None, False, True, 0, 1, 0.0, 0.1,
                     "", "non-empty", [], [""],
                     (), (0,), {}, {"a": 1}])
list(res) # 保留Stream中的truthy值

[True, 1, 0.1, 'non-empty', [''], (0,), {'a': 1}]

## ~~`every/some`~~

## reject

In [54]:
list(iters.reject(X%3==0, [2,3,4,5,6]))

[2, 4, 5]

In [55]:
list(iters.filterfalse(X%3==0, [2,3,4,5,6]))

[2, 4, 5]

In [56]:
list(iters.filter(X%3==0, [2,3,4,5,6]))

[3, 6]

In [57]:
print(iters.filterfalse == iters.reject) # weird?

False


In [58]:
list(iters.reject(None, [2,3,4,5,6])) # fn=None 表示去掉所有的Truthy值

[]

## iterate

In [59]:
fn = X**2
my_iterator = iters.iterate(fn, 2)
list(iters.take(5, my_iterator)) # iters.iterate相当于 fn(fn(fn(?))) ...

[2, 4, 16, 256, 65536]

## padNone

In [60]:
list(iters.take(5, iters.padnone([8,5])) )

[8, 5, None, None, None]

## ncycles

In [61]:
list(iters.ncycles([2,3,9], 2))

[2, 3, 9, 2, 3, 9]

## repeatFunc

In [62]:
f = lambda : 'result'
list(iters.repeatfunc(f, times=5))

['result', 'result', 'result', 'result', 'result']

## grouper

In [63]:
list(iters.grouper(3, 'abcdefg', fillvalue='NaN'))

[('a', 'b', 'c'), ('d', 'e', 'f'), ('g', 'NaN', 'NaN')]

## group_by

In [64]:
keyfunc = len
iters.group_by(keyfunc, ['Able', 'Baker', 'Charly', 'Dog', 'Emily'])

{3: ['Dog'], 4: ['Able'], 5: ['Baker', 'Emily'], 6: ['Charly']}

## round_robin

In [65]:
family1 = ['a1','a2','a3']
family2 = ['b1','b2','b3','b4']
family3 = ['c1','c2']
list(iters.roundrobin(family1, family2, family3))

['a1', 'b1', 'c1', 'a2', 'b2', 'c2', 'a3', 'b3', 'b4']

## partition

In [66]:
falsy, truthy = iters.partition(X%2==0, range(10))
list(falsy), list(truthy)

([1, 3, 5, 7, 9], [0, 2, 4, 6, 8])

## splitat

In [67]:
before, after = iters.splitat(2, 'abcde')
list(before), list(after)

(['a', 'b'], ['c', 'd', 'e'])

## splitby

In [68]:
truthy, falsy = iters.splitby(X%2==0, range(10)) # 遇到第一个falsy就切开
list(truthy), list(falsy)

([0], [1, 2, 3, 4, 5, 6, 7, 8, 9])

## powerset

In [69]:
list(iters.powerset([1,2,'A'])) # 所有子集

[(), (1,), (2,), ('A',), (1, 2), (1, 'A'), (2, 'A'), (1, 2, 'A')]

## pairwise

In [70]:
list(iters.pairwise('abcdefg')) # 相邻元素组成2gram

[('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e'), ('e', 'f'), ('f', 'g')]

## iter_except

In [71]:
d = [1,2,3,4]
my_iterator = iters.iter_except(d.pop, IndexError) # call左边的函数，并将其返回值插入到流;直到catch了右边的Exception
list(my_iterator)

[4, 3, 2, 1]

## flatten 支持多级展开

In [72]:
arr = [1, [2,3], [4,[5,[6,7]]]]
list(iters.flatten(arr))

[1, 2, 3, 4, 5, 6, 7]

## accumulate

In [73]:
list(iters.accumulate([1,2,3,4,5], operator.mul))

[1, 2, 6, 24, 120]

# Stream

In [74]:
from fn import Stream

## slice recursive

In [75]:
s = Stream() << range(10)
list(s[2:9][:3]) # s[2:9]是[2,3,4,5,6,7,8]，对这个Stream再取[:3]就是[2,3,4]

[2, 3, 4]

## fibonacci

In [76]:
s = Stream() << [0,1]
fib = s << iters.map(operator.add, s, iters.drop(1,s)) # 注意最右边的<<是一直在往Stream里面插入新值，导致s的cursor往后走
list(iters.take(5, fib))

[0, 1, 1, 2, 3]

# Option / Monad

## create option

In [77]:
from fn import monad

In [78]:
monad.Option(10, checker=X>70)

Empty()

In [79]:
monad.Empty(5)

Empty()

In [80]:
monad.Full(10)

Full(10)

In [81]:
monad.Option.from_value(10)

Full(10)

In [82]:
monad.Option.from_call(X**2, 5)

Full(25)

In [83]:
monad.Option.from_call(X[X], dict(k='v'), 'bad_k', exc=KeyError)

Empty()

## monad auto flatten

In [84]:
monad.Option(monad.Option(monad.Option(10)))

Full(10)

## map/filter

In [85]:
class Request(dict): # 注意继承
    def param(self, name):
        return monad.Option(self.get(name, None)) # 注意这里，手动包了一层monad.Option
    
    # 注意，这个函数跟上面功能相同；但是自动被包裹了一层 Option；不用自己写monad.Option(...)；可读性更好，且对旧代码改动小
    @monad.optionable
    def optParam(self, name):
        return self.get(name, None)
    
req = Request(testing='Fixed', empty='')

In [86]:
(req.param('testing') # 拿到monad参数
 .map(operator.methodcaller('strip')) # 去掉首尾空白字符
 .filter(len) # 去掉空串
 .map(operator.methodcaller('upper')) # 转大写
 .get_or('empty_value') # 取出值来
)

'FIXED'

In [87]:
(req.optParam('testing') # optParam函数用的是注解式写法
 .map(operator.methodcaller('strip')) 
 .filter(len) 
 .map(operator.methodcaller('upper')) 
 .get_or('empty_value')
)

'FIXED'

In [88]:
(req.param('empty') 
 .map(operator.methodcaller('strip')) 
 .filter(len) # 去掉空串, 变成Monad(None)
 .map(operator.methodcaller('upper'))
 .get_or('empty_value')
)

'empty_value'

In [89]:
(req.param('missing') # missing字段不存在，拿到的是Monad.Empty
 .map(operator.methodcaller('strip'))
 .filter(len)
 .map(operator.methodcaller('upper'))
 .get_or('empty_value')
)

'empty_value'

## orcall

In [90]:
# 可以直接返回值（None）表示Empty
from_mimetype = X.call('get', 'mimetype', None)

# 也可以返回 Monad
def from_extension(r):
    return (monad.Option(r.get('url',None))
            .map(X.call('split','.')[-1])
           )

In [91]:
r = dict(type='jpeg')

(monad.Option(r.get('type', None)) # 直接得到
 .or_call(from_mimetype, r)
 .or_call(from_extension, r)
 .map(operator.methodcaller('upper'))
 .get_or('unknown')
)

'JPEG'

In [92]:
r = dict(mimetype='png')

(monad.Option(r.get('type', None)) # Monad.Empty
 .or_call(from_mimetype, r) # 得到 Monad('png')
 .or_call(from_extension, r)
 .map(operator.methodcaller('upper'))
 .get_or('unknown')
)

'PNG'

In [93]:
r = dict(url='image.gif')

(monad.Option(r.get('type', None)) # Monad.Empty
 .or_call(from_mimetype, r) 
 .or_call(from_extension, r) # 'gif'
 .map(operator.methodcaller('upper'))
 .get_or('unknown')
)

'GIF'

In [94]:
r = dict()

(monad.Option(r.get('type', None)) 
 .or_call(from_mimetype, r) 
 .or_call(from_extension, r) # Monad.Empty
 .map(operator.methodcaller('upper'))
 .get_or('unknown')
)

'unknown'

# Trampolines (TCO)

In [95]:
import sys
sys.getrecursionlimit()

1000

In [ ]:
!pip install memory_profiler
%load_ext memory_profiler

In [ ]:
%%time

def factorial_normal_recurive(n): # 最直观的写法， f(n) = f(n-1) * n
    if n==1:
        return 1
    else:
        return factorial_normal_recurive(n-1) * n

%memit factorial_normal_recurive(900)
# %mprun -f factorial_normal_recurive factorial_normal_recurive(900) # mprun只能用在真实文件上，不能用在IPython中定义的函数上

In [ ]:
%%time

def factorial_tail_recursive(n, acc=1): # 尾递归，把子递归的结果直接返回； f(n,acc) = f(n-1, acc*n)
    if n==1:
        return acc
    else:
        return factorial_tail_recursive(n-1, acc*n)
    
%memit factorial_tail_recursive(900)

In [ ]:
%%time

from fn import recur

@recur.tco
def factorial_tco(n, acc=1): # 跟尾递归同理，但是写成了指定的格式；便于开启TCO自动优化
    if n==1:
        return False, acc
    else:
        return True, (n-1, acc*n)
    
%memit factorial_tco(90000) # 注意，这里的尾调用已经被优化成了循环，所以可以加大深度

# SkewHeap / PairingHeap

## basic usage

In [ ]:
from fn.immutable import SkewHeap, PairingHeap, LinkedList, Stack, Queue, Vector, Deque

In [ ]:
heap = SkewHeap()
for i in [9,3,5,2]:
    heap = heap.insert(i) # 注意是Immutable，不能inplace改
    
list(heap) # 不指定cmp时，用的是自然排序

In [ ]:
ele, remain = heap.extract()
ele, list(remain)

## specify key/cmp

In [ ]:
from collections import namedtuple
Student = namedtuple('Student', ['age','name'])
s1 = Student(7, 'Tora')
s2 = Student(5, 'Ponyo')
s3 = Student(6, 'Sousuke')

heap = SkewHeap(key=operator.attrgetter('age')) # 指定key
for s in [s1,s2,s3]:
    heap = heap.insert(s)
    
list(heap)

In [ ]:
heap = SkewHeap(cmp=X.age-X.age) # 指定cmp
for s in [s1,s2,s3]:
    heap = heap.insert(s)
    
list(heap)

In [ ]:
ph = PairingHeap() # 支持存Pair的堆
ph = ph.insert((15,'a'))
ph = ph.insert((12,'b'))
ph = ph.insert((13,'c'))

list(ph)

# LinkedList, Vector, etc.

fn 提供了这些结构的Immutable版本；用得不多，暂不了解